In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# 모든 document를 저장할 수 있는 text_set 선언
text_set = [] 

from collections import defaultdict

text_dict = defaultdict(list) # 각 text의 내용과 index를 저장하는 defaultdict

# open the folder where documents are saved, and save all the documents
path = 'data/'

for filename in os.listdir(path):
    file = open(os.path.join(path,filename), 'r', encoding='utf-8')
    text = file.readlines() # 내용을 읽는다. 
    filename_int = int(filename.replace(".txt", "")) # 파일 이름에서 .txt를 지우고 index값을 int로 바꾼다. 
    text_dict[filename_int] = text 
    #text_set.append(text)

In [3]:
text_dict.keys()

dict_keys([0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 6, 7, 8, 9])

In [4]:
from collections import OrderedDict

ordered_text_dict = OrderedDict(sorted(text_dict.items())) # index별로 sort한다. 

In [5]:
ordered_text_dict.keys() # 제대로 sort됬는지 본다. 

odict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [6]:
for text in ordered_text_dict: #text_set에 넣는다. 
    text_set.append(ordered_text_dict[text])

In [7]:
text_set[0][0] # 체크한다. 

' (CNN)If you remember Miley twerking or Kanye\'s mic-snatching, you\'ll know music awards can get chaotic.Like on Tuesday, when singer Taylor Swift took a Twitter dig at hip-hop queen Nicki Minaj and the Internet erupted into a sprawling conversation about race and gender. And then Kim Kardashian got involved. Stay with us.\'Black women are rarely rewarded\'It started when Minaj took to her keyboard to vent after one of her biggest hits wasn\'t nominated for video of the year for the MTV Video Music Awards.Mind you, this isn\'t any old clip. "Anaconda," an uproarious, raunchy twist on pop culture portrayals of curvy black women, received 19.6 million clicks in its first 24 hours, a music video record at the time. So the rapper had a reason to be upset when she was snubbed -- and she made her reasoning clear."If I was a different \'kind\' of artist, Anaconda would be nominated for best choreo and vid of the year as well," said Minaj."Black women influence pop culture so much but are ra

# TF-IDF Flow

- 1. 각 text별로 tokenize를 한다. 
- 2. token에서 stopwords나 필요없는 부분을 제거하고 stemming을 해준다. 
- 3. 각 document, 각 token별 token 몇개 있는지 구한 다음, table로 저장한다. 
- 4. 각 token별 어떤 document가 있는 지 inverted index를 구한다. 
- 5. tf-idf를 구한다.
- 6. query가 속한 document들만 가져온다. 
- 7. query의 tf-idf를 구한다. 
- 8. cosine-similarity를 구한다. 

# NLTK Preprocessing

### 1. 각 text별로 tokenize를 한다. 
### 2. token에서 stopwords나 필요없는 부분을 제거하고 stemming을 해준다. 

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [9]:
# stopwords 불러오기.
stop = set(stopwords.words('english'))

In [10]:
# 영어 소문자만 걸러내기 위한 regex
regex = re.compile('^[a-z]*[a-z]$')

In [11]:
regex.match('president')

<re.Match object; span=(0, 9), match='president'>

In [12]:
vocab_set = set() # 모든 단어의 set
vocab_list = [] # 각 document별로 단어.
porter_stemmer = PorterStemmer() # stemmer

#for i in range(len(text_set)):
for i in range(len(text_set)):
    tokenized_temp = word_tokenize(text_set[i][0].lower()) # 소문자로 만든 뒤 tokenized를 했다. 
    split_tokenized = [] # hypen이나 .으로 연결되어 있는 token들을 포함하여 전체적인 token을 일시적으로 포함하는 list.
        
    for token in tokenized_temp:
        if '-' in token: # '-'으로 연결되어 있으면 나눠준다. 
            split_tokens = token.split('-')
            for split_token in split_tokens:
                split_tokenized.append(split_token)
        elif '.' in token: # '.'으로 연결되어 있으면 나눠준다. 
            split_tokens = token.split('.')
            for split_token in split_tokens:
                split_tokenized.append(split_token)
        else: # 해당되지 않으면 넣는다. 
            split_tokenized.append(token)
    
    
    # stopwords가 아닌것과 알파벳인것만 vocab_list에 넣는다. 
    vocab_list.append([porter_stemmer.stem(token) for token in split_tokenized if token not in stop and regex.match(token)])
    
    
    # 각 vocab을 vocab_set에 넣는다. 
    for vocab in vocab_list[i]:
        vocab_set.add(vocab)

In [13]:
tokenized_temp = word_tokenize(text_set[30][0].lower())

In [14]:
vocab_list[0]

['cnn',
 'rememb',
 'miley',
 'twerk',
 'kany',
 'mic',
 'snatch',
 'know',
 'music',
 'award',
 'get',
 'chaotic',
 'like',
 'tuesday',
 'singer',
 'taylor',
 'swift',
 'took',
 'twitter',
 'dig',
 'hip',
 'hop',
 'queen',
 'nicki',
 'minaj',
 'internet',
 'erupt',
 'sprawl',
 'convers',
 'race',
 'gender',
 'kim',
 'kardashian',
 'got',
 'involv',
 'stay',
 'us',
 'women',
 'rare',
 'start',
 'minaj',
 'took',
 'keyboard',
 'vent',
 'one',
 'biggest',
 'hit',
 'nomin',
 'video',
 'year',
 'mtv',
 'video',
 'music',
 'award',
 'mind',
 'old',
 'clip',
 'anaconda',
 'uproari',
 'raunchi',
 'twist',
 'pop',
 'cultur',
 'portray',
 'curvi',
 'black',
 'women',
 'receiv',
 'million',
 'click',
 'first',
 'hour',
 'music',
 'video',
 'record',
 'time',
 'rapper',
 'reason',
 'upset',
 'snub',
 'made',
 'reason',
 'clear',
 'differ',
 'artist',
 'anaconda',
 'would',
 'nomin',
 'best',
 'choreo',
 'vid',
 'year',
 'well',
 'said',
 'minaj',
 'black',
 'women',
 'influenc',
 'pop',
 'cultur'

In [15]:
len(vocab_set)

4091

In [16]:
len(vocab_list)

60

# Matrix

### 3. 각 document, 각 token별 token 몇개 있는지 구한 다음, table로 저장한다. 
### 4. 각 token별 어떤 document가 있는 지 inverted index를 구한다. 

In [17]:
from collections import defaultdict
import collections

doc_dictionary_list = [] # 각 document별 각 word별로 몇 개 있는 지를 저장하는 dictionary를 모아놓은 list

for i in range(len(vocab_list)):    
    doc_dictionary = defaultdict(int) # 각 document별로 word가 몇 개 있는 지 저장되어 있다. 

    for vocab in vocab_list[i]: # vocab_list에 단어가 몇개가 있는 지 저장 한다. 
        doc_dictionary[vocab] += 1

    for vocab in vocab_set: # 그리고 나머지는 다 0으로 채운다. 
        if doc_dictionary[vocab] != 0:
            pass
    
    od = collections.OrderedDict(sorted(doc_dictionary.items())) # keyword들을 sort한다. 
    
    doc_dictionary_list.append(od) # 마지막으로 lsit에 저장한다. 

In [18]:
inverted_index_dic = defaultdict(list) # inverted_index를 포함하는 dictionary

for vocab in vocab_set: # 만약 doc_dictionary에서 그 단어가 1이상 나오면 그 document의 index를 저장한다. 
    index = 0
    for doc_dictionary in doc_dictionary_list:
        if doc_dictionary[vocab] >= 1:
            inverted_index_dic[vocab].append(index)
    
        index += 1

In [19]:
doc_dictionary_list[0]

OrderedDict([('aaron', 0),
             ('aback', 1),
             ('abandon', 0),
             ('abdu', 0),
             ('abel', 0),
             ('abil', 0),
             ('abl', 0),
             ('abroad', 0),
             ('absenc', 0),
             ('absente', 0),
             ('absolut', 0),
             ('abstain', 0),
             ('abstent', 0),
             ('absurd', 1),
             ('academi', 0),
             ('accept', 0),
             ('access', 0),
             ('accid', 0),
             ('acclaim', 0),
             ('accommod', 0),
             ('accompani', 0),
             ('accomplish', 0),
             ('accord', 1),
             ('accordingli', 0),
             ('account', 0),
             ('accru', 0),
             ('accur', 0),
             ('accus', 1),
             ('ace', 0),
             ('achiev', 0),
             ('acknowledg', 0),
             ('acosta', 0),
             ('acquir', 0),
             ('across', 0),
             ('act', 0),
             ('

In [20]:
inverted_index_dic

defaultdict(list,
            {'g': [18, 57],
             'review': [57],
             'list': [9, 10, 12, 19, 21, 24, 27, 39, 52, 59],
             'attribut': [18],
             'mere': [21, 49, 58],
             'turkish': [46],
             'born': [18, 20, 39],
             'unconvent': [30],
             'today': [6, 18, 36, 39],
             'jolla': [31],
             'mark': [23, 29, 34, 35, 49, 53],
             'waynegroup': [18],
             'trip': [22, 23, 29, 31, 32, 50],
             'consecut': [10, 23, 29, 32],
             'fenc': [53],
             'brown': [4, 8],
             'trap': [47],
             'trailer': [18],
             'obamacar': [47, 50, 54],
             'illustri': [31],
             'uptown': [4],
             'pound': [47, 48],
             'coordin': [47],
             'ail': [6],
             'emir': [37, 38],
             'hunt': [31, 34],
             'veto': [44],
             'temper': [51],
             'upset': [0, 44],
             'a

# tf-idf calcluation

### 5. tf-idf를 구한다.

In [81]:
array = list()

for value in doc_dictionary_list[0].values(): # 제일 첫번째 같은 경우 numpy_array_stack에다가 저장한다. 
    array.append(value)

numpy_array_stack = np.array(array) 

for i in range(1, len(doc_dictionary_list)):#1번부터 60번까지 하나씩 numpy_array_stack에 저장한다. 
    array = list()
    
    for value in doc_dictionary_list[i].values():
        array.append(value)

    numpy_array = np.array(array)
    
    numpy_array_stack = np.vstack((numpy_array_stack, numpy_array))

In [22]:
# frequency table 
numpy_array_stack

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
df = np.sum(numpy_array_stack, axis=0)
N = len(numpy_array_stack)
tf = numpy_array_stack + 1

In [24]:
# tf_idf를 구한다. 
tf_idf = np.log(tf + 1) * np.log(N / df)

# query

### 6. query가 속한 document들만 가져온다. 

In [25]:
with open('query.txt', 'r') as f:
    query = f.readline()

In [26]:
query

'president obama'

In [27]:
query_word = query.split(' ') # query문을 token별로 자른다. 

In [42]:
for i in range(len(query_word)): # 각 query word에 stemming을 해준다. 
    query_word[i] = porter_stemmer.stem(query_word[i])

In [44]:
print(query_word)

['presid', 'obama']


In [45]:
set_presi = set(inverted_index_dic[query_word[0]]) # presi가 있는 document의 index를 구한다. 
set_obama = set(inverted_index_dic[query_word[1]]) # obama가 있는 document의 index를 구한다. 

In [46]:
print(set_presi, set_obama)

{6, 7, 9, 14, 17, 30, 36, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59} {36, 6, 40, 41, 43, 44, 46, 47, 48, 49, 50, 53, 54, 57, 58}


In [47]:
query_doc_set = set() # 두 query가 동시에 들어가 있는 set을 구한다. 
query_doc_set = set_presi & set_obama

In [48]:
print(query_doc_set)

{36, 6, 40, 41, 43, 46, 47, 48, 49, 50, 53, 54, 57, 58}


In [49]:
len(query_doc_set)

14

In [50]:
query_doc_set

{6, 36, 40, 41, 43, 46, 47, 48, 49, 50, 53, 54, 57, 58}

In [51]:
print(doc_dictionary_list[6]['obama'], doc_dictionary_list[6]['presid'])

4 2


In [96]:
[i for i,t in enumerate(doc_dictionary_list[0]) if t == 'obama']

[2457]

In [97]:
[i for i,t in enumerate(doc_dictionary_list[0]) if t == 'presid']

[2739]

In [98]:
[i for i,t in enumerate(doc_dictionary_list[0]) if t == 'aaron']

[0]

In [99]:
for index in query_doc_set:
    print(index)
    print(tf_idf[index][2457], tf_idf[index][2739])

36
0.05635144354043394 -0.6355299302838787
6
0.09190524592679582 -0.8019494846025861
40
0.05635144354043394 -0.9310345194895925
41
0.07110760477272376 -1.638969359436466
43
0.07110760477272376 -0.6355299302838787
46
0.1274590483131577 -0.8019494846025861
47
0.1274590483131577 -1.2710598605677574
48
0.09190524592679582 -1.1256784164203144
49
0.07110760477272376 -1.3871447017920355
50
0.10666140715908563 -0.9310345194895925
53
0.082553372640967 -0.8019494846025861
54
0.082553372640967 -1.3871447017920355
57
0.05635144354043394 -1.0365046725851716
58
0.05635144354043394 -1.3320092617908856


# Cosine similarity

https://comscience.tistory.com/entry/TF-IDF와-consine-similarity


https://wikidocs.net/24603